In [6]:
# importing the dependencies. 
import pandas as pd
import numpy as np
import h5py
import obspy
from obspy.signal.filter import envelope
from tqdm import tqdm
import warnings
import argparse
import sys
import os
from scipy.signal import resample_poly
from scatseisnet import ScatteringNetwork
import matplotlib.dates as mdates
import pickle
from scipy.signal import butter, filtfilt
from scipy.signal import butter, lfilter


from glob import glob

In [3]:
segment_duration_seconds = start_time+end_time
sampling_rate_hertz = new_sr
samples_per_segment = int(segment_duration_seconds * sampling_rate_hertz)
bank_keyword_arguments = (
    {"octaves": 5, "resolution": 2, "quality": 1},
    {"octaves": 5, "resolution": 2, "quality": 3},
)

In [4]:
network = ScatteringNetwork(
    *bank_keyword_arguments,
    bins=samples_per_segment,
    sampling_rate=sampling_rate_hertz,
)

In [50]:
new_su_cat

,file_path,extracted_value,trace_start_time,event_id,snr,station_code,station_network_code,station_channel_code,station_location_code
4,../data/surface_event_waveforms/CC.VALT..BHN__...,20070327T042554Z,2007-03-27T04:27:04.000000Z,pnsn1002133,13.662326,VALT,CC,BHN,NaN
7,../data/surface_event_waveforms/CC.SWFL..BHN__...,20070128T221646Z,2007-01-28T22:17:56.000000Z,pnsn1003458,21.925673,SWFL,CC,BHN,NaN
10,../data/surface_event_waveforms/CC.VALT..BHN__...,20070331T043435Z,2007-03-31T04:35:45.000000Z,pnsn1003743,15.829949,VALT,CC,BHN,NaN
13,../data/surface_event_waveforms/CC.VALT..BHN__...,20070802T102620Z,2007-08-02T10:27:30.000000Z,pnsn1004993,10.552246,VALT,CC,BHN,NaN
16,../data/surface_event_waveforms/CC.VALT..BHN__...,20070710T022546Z,2007-07-10T02:26:56.000000Z,pnsn1005503,13.624090,VALT,CC,BHN,NaN
...,...,...,...,...,...,...,...,...,...
52333,../data/surface_event_waveforms/UW.LON..BHN__2...,20060830T062501Z,2006-08-30T06:26:11.000000Z,pnsn990623,13.813825,LON,UW,BHN,NaN
52357,../data/surface_event_waveforms/UW.LON..BHN__2...,20061130T041335Z,2006-11-30T04:14:45.000000Z,pnsn991783,14.199901,LON,UW,BHN,NaN
52369,../data/surface_event_waveforms/UW.LON..BHN__2...,20070501T000653Z,2007-05-01T00:08:03.000000Z,pnsn993093,10.803047,LON,UW,BHN,NaN
52384,../data/surface_event_waveforms/UW.LON..BHN__2...,20070601T225309Z,2007-06-01T22:54:19.000000Z,pnsn993443,12.697431,LON,UW,BHN,NaN


In [47]:
# Constants
START_TIME = 10
END_TIME = 30
NEW_SR = 50
ORIG_SR = 100
TOTAL_SAMPLES = int((START_TIME + END_TIME) * ORIG_SR)

# Load dataset
new_su_cat = pd.read_csv(glob('../../data/additional_surface_events_good_snr.csv')[0], index_col=0)

# Initialize data storage
data = []
evids = []


# Process each event
for idx in tqdm(range(len(new_su_cat)), desc="Processing Events"):
    ticker = new_su_cat['extracted_value'].iloc[idx]
    event_id = new_su_cat['event_id'].iloc[idx]
    stn_code = new_su_cat['station_code'].iloc[idx]
    net_code = new_su_cat['station_network_code'].iloc[idx]
    
    
    files = glob(f'../../data/surface_event_waveforms/*{ticker}*.mseed')

    # Ensure all 3 components exist
    if len(files) == 3:
        d_array = np.zeros((3, TOTAL_SAMPLES))

        for j, file in enumerate(files):
            tr = obspy.read(file)[0]
            tr.detrend('linear')
            tr.resample(ORIG_SR)  # Ensuring consistent sample rate
            d_array[j, :] = tr.data[7000 - START_TIME * ORIG_SR : 7000 + END_TIME * ORIG_SR]
            
        evids.append(event_id+'_'+net_code+'.'+stn_code)
        data.append(d_array)

# Convert list to NumPy array
data = np.array(data)
evids = np.array(evids)

Processing Events: 100%|██████████| 6517/6517 [10:41<00:00, 10.16it/s]


In [41]:
import numpy as np
import pandas as pd
import obspy
from glob import glob
from tqdm import tqdm

# Constants
START_TIME = 10
END_TIME = 30
ORIG_SR = 100
TOTAL_SAMPLES = (START_TIME + END_TIME) * ORIG_SR

# Load dataset
new_su_cat = pd.read_csv(glob('../../data/additional_surface_events_good_snr.csv')[0], index_col=0)

# Initialize data storage
data = []
evids = []

# Process each event using itertuples() for faster iteration
for row in tqdm(new_su_cat.itertuples(index=False), total=len(new_su_cat), desc="Processing Events"):
    files = glob(f'../../data/surface_event_waveforms/*{row.extracted_value}*.mseed')

    # Ensure all 3 components exist
    if len(files) == 3:
        try:
            # Preallocate array
            d_array = np.zeros((3, TOTAL_SAMPLES))

            for j, file in enumerate(files):
                tr = obspy.read(file)[0]  # Read one file at a time
                tr.detrend('linear')

                # Only resample if needed
                if tr.stats.sampling_rate != ORIG_SR:
                    tr.resample(ORIG_SR)

                # Extract segment
                d_array[j, :] = tr.data[7000 - START_TIME * ORIG_SR : 7000 + END_TIME * ORIG_SR]

            # Store processed event
            evids.append(f"{row.event_id}_{row.station_network_code}.{row.station_code}")
            data.append(d_array)

        except Exception as e:
            print(f"Error processing {row.extracted_value}: {e}")

# Convert lists to NumPy arrays for fast numerical operations
data = np.array(data)
evids = np.array(evids)


Processing Events: 100%|██████████| 6517/6517 [10:42<00:00, 10.15it/s]


In [56]:
import json
data_path = '../../data/new_curated_surface_event_data.npy'
ids_path = '../../data/new_curated_surface_event_ids.json'


# Load data and IDs
surface_data = np.load(data_path, allow_pickle=True)
with open(ids_path, "r") as file:
    surface_ids = json.load(file)

In [59]:
surface_data

array([array([[  3.52544727,   7.43295676,  12.35178909, ...,  -5.51723434,
                -1.52544727,   0.87720287],
              [  2.72202467,   5.10447253,   5.63809868, ..., -16.20388239,
                -8.72202467,  -1.9377051 ],
              [  5.18385376,   9.80662742,  13.73541505, ...,   5.22294519,
                 2.31614624,   2.20739083]])                               ,
       array([[ 0.75536171,  1.73481689,  3.02144683, ..., -0.38628986,
               -0.50536171, -0.02614417],
              [-0.49858318, -1.47652756, -2.4943327 , ...,  2.53364391,
                1.49858318,  0.42440854],
              [-1.75138904, -4.04558796, -7.00555617, ...,  1.53845516,
                0.50138904, -0.36073136]])                             ,
       array([[ 0.74688854,  1.15537278,  1.23755417, ...,  3.836684  ,
                2.00311146,  0.84517625],
              [ 0.75136126,  2.46542344,  5.50544505, ...,  1.57266699,
                0.99863874,  0.46540516],
      

In [73]:
low = 0.5
high = 15
taper_amount = 10
sampling_rate = 100
original_rate = 100
desired_rate = 50
original_sr = 100

def cosine_taper_bandpass_filter(data, sampling_rate, lowcut = low, highcut = high, taper_amount = taper_amount):
    # Define the percentage of tapering (10%)
    taper_percentage = taper_amount/100

    # Create a cosine taper window
    taper_length = int(len(data) * taper_percentage)
    taper = 0.5 * (1 - np.cos(2 * np.pi * np.arange(taper_length) / (taper_length - 1)))

    # Apply the taper to the beginning and end of the data
    data[:taper_length] *= taper
    data[-taper_length:] *= taper[::-1]


    # Normalize the cutoff frequencies
    lowcut /= (0.5 * sampling_rate)
    highcut /= (0.5 * sampling_rate)

    # Design the Butterworth bandpass filter
    b, a = butter(4, [lowcut, highcut], btype='band')

    # Apply the bandpass filter to the data using filtfilt
    filtered_data = filtfilt(b, a, data)

    return filtered_data



def normalize_arrays(array_of_arrays):
    normalized_arrays = []
    
    for sub_arrays in array_of_arrays:
        normalized_sub_arrays = []
        for arr in sub_arrays:
            max_val = np.max(np.abs(arr))
            normalized_arr = arr / max_val if max_val != 0 else arr  # Avoid division by zero
            normalized_sub_arrays.append(normalized_arr)
        normalized_arrays.append(normalized_sub_arrays)
    
    return np.array(normalized_arrays)



def resample_arrays_to_rate(array_of_arrays, original_rate, desired_rate):
    resampled_arrays = []
    gcd = np.gcd(original_rate, desired_rate)  # Greatest common divisor
    up = desired_rate // gcd  # Upsampling factor
    down = original_rate // gcd  # Downsampling factor
    
    for sub_arrays in array_of_arrays:
        resampled_sub_arrays = []
        for arr in sub_arrays:
            resampled_arr = resample_poly(arr, up, down)
            resampled_sub_arrays.append(resampled_arr)
        resampled_arrays.append(resampled_sub_arrays)
    
    return np.array(resampled_arrays)




In [85]:
data = []
new_ids = []
for i in tqdm(range(len(surface_data))):
    if np.shape(surface_data[i]) == (3, 18000):
        data.append(surface_data[i][:, int((70 - start_time)*orig_sr): int((70+end_time)*original_sr)])
        new_ids.append(surface_ids[i])

100%|██████████| 6495/6495 [00:00<00:00, 506950.60it/s]


In [89]:
filtered_data = np.zeros_like(data)
for i in tqdm(range(len(data))):
    for j in range(len(data[i])):
        filtered_data[i][j] = cosine_taper_bandpass_filter(data[i][j], original_sr)

100%|██████████| 6487/6487 [00:11<00:00, 550.86it/s]


In [90]:
norm_array = normalize_arrays(filtered_data)
resampled_array = resample_arrays_to_rate(norm_array, original_sr, new_sr)

In [96]:
output_path = '/home/ak287/PNW_Seismic_Event_Classification/extracted_features'
scattering_coefficients = []
serial_ids = []
part = 1
for i in tqdm(range(len(resampled_array))):
    segments = resampled_array[i]
    scattering_coefficients.append(network.transform(segments, reduce_type=np.max))
    serial_ids.append(new_ids[i])

order_1 = []
order_2 = []
for i in range(len(scattering_coefficients)):
    order_1.append(scattering_coefficients[i][0][2])
    order_2.append(scattering_coefficients[i][1][2])
    
order_1 = np.array(order_1)
order_2 = np.array(order_2)
    
np.savez(
    os.path.join(output_path, f'scattering_coefficients_new_surface_events_P_{start_time}_{end_time}_F_{int(low)}_{int(high)}_{new_sr}_part_{part}.npz'),
    order_1 = order_1,
    order_2 = order_2,
    serial_ids = serial_ids
)


100%|██████████| 6487/6487 [01:13<00:00, 87.74it/s]


(6487, 10)

In [94]:
order_2.shape

(6487, 10, 10)

In [103]:
a = np.load('../../extracted_features/scattering_coefficients_surface event_P_10_30_F_0_15_50_part_1.npz')

In [106]:
a['order_2'].shape

(8912, 10, 10)

In [107]:
# Import dependencies
import os
import json
import numpy as np
import pandas as pd
import obspy
import h5py
import pickle
import warnings
import argparse
import sys
import matplotlib.dates as mdates
from tqdm import tqdm
from scipy.signal import butter, filtfilt, resample_poly
from glob import glob
from obspy.signal.filter import envelope
from scatseisnet import ScatteringNetwork

# Constants
START_TIME = 10
END_TIME = 30
LOWCUT = 0.5
HIGHCUT = 15
TAPER_AMOUNT = 10
ORIGINAL_SR = 100
NEW_SR = 50
SEGMENT_DURATION = START_TIME + END_TIME
SAMPLES_PER_SEGMENT = int(SEGMENT_DURATION * NEW_SR)
OUTPUT_PATH = '/home/ak287/PNW_Seismic_Event_Classification/extracted_features'

# Bandpass filter with cosine taper
def cosine_taper_bandpass_filter(data, sr, lowcut=LOWCUT, highcut=HIGHCUT, taper_amount=TAPER_AMOUNT):
    taper_len = int(len(data) * (taper_amount / 100))
    taper = 0.5 * (1 - np.cos(2 * np.pi * np.arange(taper_len) / (taper_len - 1)))

    # Apply taper
    data[:taper_len] *= taper
    data[-taper_len:] *= taper[::-1]

    # Design and apply Butterworth filter
    nyq = 0.5 * sr
    b, a = butter(4, [lowcut / nyq, highcut / nyq], btype='band')
    return filtfilt(b, a, data)

# Normalize arrays
def normalize_arrays(arr):
    max_vals = np.max(np.abs(arr), axis=-1, keepdims=True)
    return np.where(max_vals != 0, arr / max_vals, arr)

# Resample arrays
def resample_arrays(arr, orig_sr, new_sr):
    gcd = np.gcd(orig_sr, new_sr)
    up, down = new_sr // gcd, orig_sr // gcd
    return np.array([resample_poly(sub_arr, up, down) for sub_arr in arr])

# Initialize Scattering Network
BANK_KWARGS = ({"octaves": 5, "resolution": 2, "quality": 1}, {"octaves": 5, "resolution": 2, "quality": 3})
network = ScatteringNetwork(*BANK_KWARGS, bins=SAMPLES_PER_SEGMENT, sampling_rate=NEW_SR)

# Load data and IDs
data_path = '../../data/new_curated_surface_event_data.npy'
ids_path = '../../data/new_curated_surface_event_ids.json'
surface_data = np.load(data_path, allow_pickle=True)
with open(ids_path, "r") as file:
    surface_ids = json.load(file)

# Process data
data, new_ids = [], []
for i in tqdm(range(len(surface_data)), desc="Extracting Data"):
    if surface_data[i].shape == (3, 18000):
        data.append(surface_data[i][:, int((70 - START_TIME) * ORIGINAL_SR): int((70 + END_TIME) * ORIGINAL_SR)])
        new_ids.append(surface_ids[i])

# Convert to numpy array
data = np.array(data)

# Apply filtering and resampling
filtered_data = np.array([[cosine_taper_bandpass_filter(tr, ORIGINAL_SR) for tr in sample] for sample in tqdm(data, desc="Filtering Data")])
normalized_data = normalize_arrays(filtered_data)
resampled_data = np.array([resample_arrays(sample, ORIGINAL_SR, NEW_SR) for sample in tqdm(normalized_data, desc="Resampling Data")])

# Compute Scattering Coefficients
scattering_coeffs, serial_ids = [], []
for i in tqdm(range(len(resampled_data)), desc="Computing Scattering Coefficients"):
    scattering_coeffs.append(network.transform(resampled_data[i], reduce_type=np.max))
    serial_ids.append(new_ids[i])

# Extract orders
order_1 = np.array([coeff[0][2] for coeff in scattering_coeffs])
order_2 = np.array([coeff[1][2] for coeff in scattering_coeffs])

# Save results
np.savez(
    os.path.join(OUTPUT_PATH, f'scattering_coefficients_new_surface_events_P_{START_TIME}_{END_TIME}_F_{int(LOWCUT)}_{int(HIGHCUT)}_{NEW_SR}_part_1.npz'),
    order_1=order_1,
    order_2=order_2,
    serial_ids=serial_ids
)


Computing Scattering Coefficients: 100%|██████████| 6487/6487 [01:15<00:00, 86.35it/s]
